# 图

## 图的表示与遍历

In [4]:
class VertexNode:
    def __init__(self, id, data, first_adj = None) -> None:
        self.id = id
        self.data = data
        self.first_adj = first_adj
        
    def __repr__(self) -> str:
        return self.data
    
class EdgeNode:
    def __init__(self, vertex_id, weight = 0, next_adj = None) -> None:
        self.vertex_id = vertex_id
        self.weight = weight
        self.next_adj = next_adj
        
vertice = [VertexNode(i, f'v{i+1}', None) for i in range(8)]

vertice[0].first_adj = EdgeNode(1, 0, EdgeNode(2, 0, None))
vertice[1].first_adj = EdgeNode(0, 0, EdgeNode(3, 0, EdgeNode(4, 0, None)))
vertice[2].first_adj = EdgeNode(0, 0, EdgeNode(5, 0, EdgeNode(6, 0, None)))
vertice[3].first_adj = EdgeNode(1, 0, EdgeNode(7, 0, None))
vertice[4].first_adj = EdgeNode(1,0, EdgeNode(7, 0, None))
vertice[5].first_adj = EdgeNode(2, 0, EdgeNode(6, 0, None))
vertice[6].first_adj = EdgeNode(2, 0, EdgeNode(5, 0, None))
vertice[7].first_adj = EdgeNode(3, 0, EdgeNode(4, 0, None))


visited = [False for _ in vertice]
def dfs(vertex:VertexNode):
    visited[vertex.id] = True
    print(vertex, end=', ')
    
    edge_node = vertex.first_adj
    while edge_node != None:
        if not visited[edge_node.vertex_id]:
            dfs(vertice[edge_node.vertex_id])
            
        edge_node = edge_node.next_adj

dfs(vertice[0])

v1, v2, v4, v8, v5, v3, v6, v7, 

In [5]:
class QueueNode:
    def __init__(self, element, next=None):
        self.element = element
        self.next = next

class Queue:
    def __init__(self):
        self.front = QueueNode('Head')
        self.rear = self.front

    def enqueue(self, e):
        self.rear.next = QueueNode(e)
        self.rear = self.rear.next

    def empty(self):
        return self.front == self.rear

    def dequeue(self):
        if self.empty():
            return None
        else:
            e = self.front.next.element
            self.front.next = self.front.next.next
            if self.front.next == None:
                self.rear = self.front
            return e


In [7]:
def bfs():
    visited = [False for _ in vertice]
    q = Queue()
    q.enqueue(vertice[0])
    while not q.empty():
        current:VertexNode = q.dequeue()    
        if not visited[current.id]:
            visited[current.id] = True
            print(current, end=',')
            adj = current.first_adj
            while adj != None:
                if not visited[adj.vertex_id]:
                    q.enqueue(vertice[adj.vertex_id])
                    
                adj = adj.next_adj
                
bfs()        

v1,v2,v3,v4,v5,v6,v7,v8,

## Dijkstra算法

简化实现版本，未考虑异常情况，假设求解时，源点到目标顶点可以到达。

In [32]:
UNVISITED = 0
VISITED = 1
MAX_VALUE = 100000    

class EdgeNode:
    def __init__(self, adj_vertex_id, weight, next = None) -> None:
        """边节点"""
        self.adj_vertex_id = adj_vertex_id
        self.weight = weight
        self.next = next


class VertexNode:
    def __init__(self, label, first_adj = None) -> None:
        self.label = label
        self.first_adj = first_adj


class Graph:
    def __init__(self, vertex_labels = []) -> None:
        self.vertices = [VertexNode(label) for label in vertex_labels]
        self.marks = [UNVISITED for i in vertex_labels]
        self.edge_num = 0
        
        
    def n(self):
        """返回节点的数量"""
        return len(self.vertices)
    
    def get_mark(self, i):
        return self.marks[i]
    
    def mark(self, i, tag):
        self.marks[i] = tag
    
    def add_edge(self, i, j, weight):
        """加入一条边，未考虑重复加入的问题
        """
        vertex = self.vertices[i]
        
        if vertex.first_adj==None:
            vertex.first_adj = EdgeNode(j, weight, None)
        elif j < vertex.first_adj.adj_vertex_id:
            edgenode = EdgeNode(j, weight, vertex.first_adj)
            vertex.first_adj = edgenode
        else:
            p = vertex.first_adj
            while p.next != None and p.next.adj_vertex_id < j:
                p = p.next
            edgenode = EdgeNode(j, weight, p.next)
            p.next = edgenode
        
        self.edge_num += 1
    
    def weight(self, v, w):
        """获取边(v->w)的权重"""
        neighbor = self.vertices[v].first_adj
        while neighbor != None:
            if neighbor.adj_vertex_id == w:
                return neighbor.weight
            neighbor = neighbor.next
            
        return MAX_VALUE
    

In [36]:
G = Graph(['0', '1', '2','3','4','5','6'])

# 构建边
edges = [(0,1,2), (0,3,1), (1,3,3), (1,4,10), (2,0,4), (2,5,5), (3,2,2),(3,5,8), (3,6,4), (3,4,2), (4,6,6), (6,5,1)]
for i, j, weight in edges:
    G.add_edge(i, j, weight)
    
# 显示图的邻接表信息
for i in range(G.n()):
    print(G.vertices[i].label, end=': ')
    neighbor = G.vertices[i].first_adj
    while neighbor!=None:
        print('->', G.vertices[neighbor.adj_vertex_id].label, end=' ')
        neighbor = neighbor.next
    print()
        

0: -> 1 -> 3 
1: -> 3 -> 4 
2: -> 0 -> 5 
3: -> 2 -> 4 -> 5 -> 6 
4: -> 6 
5: 
6: -> 5 


In [37]:
def find_min_vertex(graph, D):
    # 找一个未被访问过的节点
    v = 0 
    for i in range(graph.n()):
        if graph.get_mark(i) == UNVISITED:
            v = i
            break
        
    # 寻找距离最小的未访问过的顶点
    for i in range(graph.n()):
        if graph.get_mark(i) == UNVISITED and D[i] < D[v]:
            v = i
            
    return v
        

def dijkstra(graph, source):
    """计算从原点source开始到其他位置的最短路径"""
    
    # 初始化当前节点到所有其他节点的距离为最大值
    D = [MAX_VALUE for _ in range(graph.n())]
    D[source] = 0
    for i in range(graph.n()):
        # 找到距离最近的点
        v = find_min_vertex(graph, D)
        # 标记为已经访问过
        graph.mark(v, VISITED)
        # 距离无穷大，则说明无路径可达
        if D[v] == MAX_VALUE:
            print("无法到达。。。")
            return
        #处理v的邻居，看是否通过v可以让路径变短
        neighbor = graph.vertices[v].first_adj
        while neighbor != None:
            w = neighbor.adj_vertex_id
            if D[w] > D[v] + graph.weight(v, w):
                D[w] = D[v] + graph.weight(v, w)
            neighbor = neighbor.next

    return D                
        
# 设置source为源点        
source = 0 
# source到其他顶点的距离已经保存到distances中，未考虑异常情况
distances = dijkstra(G, source)
for idx, distance in enumerate(distances):
    if idx != source:
        print(G.vertices[source].label, '到', G.vertices[idx].label, '的最短距离为：', distance)
        

0 到 1 的最短距离为： 2
0 到 2 的最短距离为： 3
0 到 3 的最短距离为： 1
0 到 4 的最短距离为： 3
0 到 5 的最短距离为： 6
0 到 6 的最短距离为： 5
